In [37]:
import pandas as pd
import numpy as np
import plotly.express as px
from src.utils import encode_frame, decode_frame, plot_climb
import torch
import plotly.graph_objects as go
from plotly.subplots import make_subplots



In [20]:
climbs = pd.read_csv("data/raw/all_climbs.csv", index_col=0)
holds = pd.read_csv("data/raw/holds.csv", index_col=0)

In [98]:
def combine_handholds(tensor):
    tensor = tensor.long()
    # Combine start, middle, and finish holds into one channel
    handholds = tensor[0] | tensor[1] | tensor[2]  # Logical OR operation
    footholds = tensor[3]
    return torch.stack([handholds, footholds])


def check_nearby(hold1, hold2, threshold=1):
    y1, x1 = torch.where(hold1)
    y2, x2 = torch.where(hold2)

    nearby_count = 0
    for y, x in zip(y1, x1):
        # Check if there's any hold in hold2 within the threshold distance
        if torch.any((torch.abs(y2 - y) <= threshold) & (torch.abs(x2 - x) <= threshold)):
            nearby_count += 1
    maxlen = max(len(y1), len(y2))
    return nearby_count / maxlen if maxlen > 0 else 0


def climb_similarity(climb1, climb2, threshold):
    combined_climb1 = combine_handholds(climb1)
    combined_climb2 = combine_handholds(climb2)

    similarity_scores = []
    for i in range(2):  # Iterate over handholds and footholds
        similarity_scores.append(check_nearby(combined_climb1[i], combined_climb2[i], threshold=threshold))

    return sum(similarity_scores) / len(similarity_scores)

In [99]:
c1 = climbs[climbs["name"] == "swooped"].iloc[0]["frames"]
c2 = climbs[climbs["name"] == "swooped hard"].iloc[0]["frames"]
c3 = climbs[climbs["name"] == "You Don’t Know Me"].iloc[0]["frames"]

In [100]:
t1, t2, t3 = encode_frame(c1, holds), encode_frame(c2, holds), encode_frame(c3, holds)

In [101]:
fig = make_subplots(rows=1, cols=3)

fig.add_trace(go.Image(z=np.asarray(plot_climb(t1)), name="swooped"), row=1, col=1)
fig.add_trace(go.Image(z=np.asarray(plot_climb(t2)), name="swooped hard"), row=1, col=2)
fig.add_trace(go.Image(z=np.asarray(plot_climb(t3)), name="You Dont know me"), row=1, col=3)
fig.update_layout(
    margin=dict(l=10, r=10, b=10, t=10),
    width=1200,  # Adjust the width as needed
    height=400,  # Adjust the height as needed
)
fig.show()

In [102]:
n = 100
subset = climbs.sort_values("ascensionist_count").drop_duplicates("frames").head(n).reset_index()

In [103]:
THRESHOLD = 1
simmat = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        t1, t2 = encode_frame(subset["frames"][i], holds), encode_frame(subset["frames"][j], holds)
        simmat[i, j] = climb_similarity(t1, t2, THRESHOLD)

In [104]:
px.imshow(simmat, width=1000, height=1000)

In [82]:
fig = make_subplots(rows=1, cols=3)

fig.add_trace(go.Image(z=np.asarray(plot_climb(encode_frame(subset["frames"][87], holds)))), row=1, col=1)
fig.add_trace(go.Image(z=np.asarray(plot_climb(encode_frame(subset["frames"][22], holds)))), row=1, col=2)
fig.add_trace(go.Image(z=np.asarray(plot_climb(encode_frame(subset["frames"][89], holds)))), row=1, col=3)

In [85]:
climb_similarity(encode_frame(subset["frames"][22], holds), encode_frame(subset["frames"][89], holds), )

0.5